<a href="https://colab.research.google.com/github/Investigator13th/CS246/blob/main/CS246_homework3_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark
!pip install -U -q PyDrive2

!mv /usr/local/lib/libtbbmalloc_proxy.so.2 /usr/local/lib/libtbbmalloc_proxy.so.2.backup
!mv /usr/local/lib/libtbbmalloc.so.2 /usr/local/lib/libtbbmalloc.so.2.backup
!mv /usr/local/lib/libtbbbind_2_5.so.3 /usr/local/lib/libtbbbind_2_5.so.3.backup
!mv /usr/local/lib/libtbb.so.12 /usr/local/lib/libtbb.so.12.backup
!mv /usr/local/lib/libtbbbind_2_0.so.3 /usr/local/lib/libtbbbind_2_0.so.3.backup
!mv /usr/local/lib/libtbbbind.so.3 /usr/local/lib/libtbbbind.so.3.backup
!ln -s /usr/local/lib/libtbbmalloc_proxy.so.2.11 /usr/local/lib/libtbbmalloc_proxy.so.2
!ln -s /usr/local/lib/libtbbmalloc.so.2.11 /usr/local/lib/libtbbmalloc.so.2
!ln -s /usr/local/lib/libtbbbind_2_5.so.3.11 /usr/local/lib/libtbbbind_2_5.so.3
!ln -s /usr/local/lib/libtbb.so.12.11 /usr/local/lib/libtbb.so.12
!ln -s /usr/local/lib/libtbbbind_2_0.so.3.11 /usr/local/lib/libtbbbind_2_0.so.3
!ln -s /usr/local/lib/libtbbbind.so.3.11 /usr/local/lib/libtbbbind.so.3

!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

openjdk-8-jdk-headless is already the newest version (8u412-ga-1~22.04.1).
The following packages were automatically installed and are no longer required:
  libhawtjni-runtime-java libjansi-java libjansi-native-java libjline2-java
  scala-parser-combinators scala-xml
Use 'apt autoremove' to remove them.
0 upgraded, 0 newly installed, 0 to remove and 47 not upgraded.


In [ ]:
import pyspark
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
from pyspark.ml.graph import GraphFrames

ModuleNotFoundError: No module named 'pyspark.ml.graph'

In [ ]:
spark = SparkSession.builder \
       .appName("Graph") \
       .config("spark.jars", "/usr/local/lib/python3.10/dist-packages/pyspark/jars/graphframes-0.8.3-spark3.5-s_2.12.jar") \
       .getOrCreate()

In [ ]:
spark = SparkSession.builder.getOrCreate()

In [ ]:
!pip install graphframes
from graphframes import GraphFrame

In [37]:
# 读取rdd
data = spark.sparkContext.textFile("/content/drive/MyDrive/hw3-bundle/q2/data/graph-full.txt")
edges_rdd = data.map(lambda line: tuple(line.split("\t")))
edges = edges_rdd.map(lambda x: (int(x[0]), int(x[1]))).distinct()

In [ ]:
# 转化为dataframe
edges_df = spark.createDataFrame(edges, ["src", "dst"]).toDF("src", "dst")
vertices_df = edges_df.select("src").union(edges_df.select("dst")).distinct().withColumnRenamed("src", "id")

In [ ]:
# 创建GraphFrame
g = GraphFrame(vertices_df, edges_df)

/usr/local/lib/python3.10/dist-packages/pyspark/sql/dataframe.py:168: UserWarning: DataFrame.sql_ctx is an internal property, and will be removed in future releases. Use DataFrame.sparkSession instead.
  warnings.warn(


In [ ]:
result = g.pageRank(resetProbability=0.2, maxIter=40)

/usr/local/lib/python3.10/dist-packages/pyspark/sql/dataframe.py:147: UserWarning: DataFrame constructor is internal. Do not directly use it.
  warnings.warn("DataFrame constructor is internal. Do not directly use it.")


In [ ]:
top_ranks = result.vertices.orderBy('pagerank', ascending=False).limit(5)
top_ranks.show()

+---+------------------+
| id|          pagerank|
+---+------------------+
|263| 2.020291181518219|
|537|  1.94334157145315|
|965| 1.925447807166263|
|243|1.8526340162417312|
|285|1.8273721700645142|
+---+------------------+



In [ ]:
bottom_ranks = result.vertices.orderBy('pagerank', ascending=True).limit(5)
bottom_ranks.show()

+---+-------------------+
| id|           pagerank|
+---+-------------------+
|558| 0.3286018525215298|
| 93|0.35135689375165774|
| 62| 0.3531481051059628|
|424|0.35481538649301453|
|408| 0.3877984871929171|
+---+-------------------+



In [34]:
sc = spark.sparkContext

In [44]:
nodes = edges.flatMap(lambda edge: edge).map(lambda node: (node, 1.0)).distinct()
# edges.join(nodes).map(lambda x: (x[1][0][1], x[1][1])).reduceByKey(lambda a, b: a + b).take(5)
edges.join(nodes).take(5)

[(816, (774, 1.0)),
 (816, (208, 1.0)),
 (816, (894, 1.0)),
 (816, (762, 1.0)),
 (816, (817, 1.0))]

In [45]:
nodes.take(5)

[(367, 1.0), (281, 1.0), (641, 1.0), (231, 1.0), (477, 1.0)]

In [57]:
import time

In [76]:
def hits(sc, edges, num_iterations=20):
    # Step 1: Create initial RDDs for hubs and authorities
    nodes = edges.flatMap(lambda edge: edge).map(lambda node: (node, 1.0)).distinct()
    authorities = nodes
    hubs = nodes

    # 缓存edges和hubs以避免重复计算

    # Step 2: Iterate over the edges to compute new scores
    for i in range(num_iterations):
        time1=time.time()

        # 更新权威值
        new_authorities = edges.join(hubs).map(lambda x: (x[1][0], x[1][1])).reduceByKey(lambda a, b: a + b)

        new_hubs = edges.map(lambda x: (x[1], x[0])).join(new_authorities).map(lambda x: (x[0], x[1][1])).reduceByKey(lambda a, b: a + b)

        # Normalize scores
        max_authorities = new_authorities.values().max()
        max_hubs = new_hubs.values().max()
        new_authorities = new_authorities.mapValues(lambda v: v / max_authorities)
        new_hubs = new_hubs.mapValues(lambda v: v / max_hubs)

        # Update authorities and hubs
        authorities = new_authorities
        hubs = new_hubs

        time2 = time.time()
        print(f"第{i+1}次迭代用时{time2-time1}秒")

    return authorities, hubs


In [77]:
# Load data

# Call the HITS function
authorities, hubs = hits(sc, edges)

# Collect results
authorities = authorities.collectAsMap()
hubs = hubs.collectAsMap()


第1次迭代用时6.413169622421265秒
第2次迭代用时12.110769033432007秒
第3次迭代用时16.838576793670654秒
第4次迭代用时23.530759811401367秒
第5次迭代用时24.924242734909058秒
第6次迭代用时31.82461190223694秒
第7次迭代用时38.88038158416748秒
第8次迭代用时41.02754187583923秒
第9次迭代用时46.601670265197754秒
第10次迭代用时52.702900648117065秒
第11次迭代用时57.86358380317688秒
第12次迭代用时65.53952646255493秒
第13次迭代用时68.47106885910034秒
第14次迭代用时71.12370705604553秒
第15次迭代用时77.64355659484863秒
第16次迭代用时84.70019221305847秒
第17次迭代用时87.32892203330994秒
第18次迭代用时93.98415756225586秒
第19次迭代用时95.3867826461792秒
第20次迭代用时101.66957759857178秒


In [78]:
top_authorities = sorted(authorities.items(), key=lambda item: item[1], reverse=True)[:5]
bottom_authorities = sorted(authorities.items(), key=lambda item: item[1], reverse=False)[:5]


top_hubs = sorted(hubs.items(), key=lambda item: item[1], reverse=True)[:5]
bottom_hubs = sorted(hubs.items(), key=lambda item: item[1], reverse=False)[:5]

print("Top 5 Authorities:")
for node, score in top_authorities:
    print(f"Node {node}: {score}")

print("\nBottom 5 Authorities:")
for node, score in bottom_authorities:
    print(f"Node {node}: {score}")

print("\nTop 5 Hubs:")
for node, score in top_hubs:
    print(f"Node {node}: {score}")

print("\nBottom 5 Hubs:")
for node, score in bottom_hubs:
    print(f"Node {node}: {score}")

Top 5 Authorities:
Node 16: 1.0
Node 537: 0.9004515846640125
Node 381: 0.8954816040952954
Node 965: 0.8933180568712675
Node 389: 0.8479288650546584

Bottom 5 Authorities:
Node 742: 0.03510360258162666
Node 93: 0.049764801616701757
Node 745: 0.053248592081665776
Node 892: 0.06153636569657152
Node 135: 0.06177882092195121

Top 5 Hubs:
Node 16: 1.0
Node 965: 0.8374856783168138
Node 537: 0.7878951365810112
Node 243: 0.7581223178745403
Node 747: 0.7475360997866405

Bottom 5 Hubs:
Node 93: 0.006220600202087722
Node 742: 0.006581925484055002
Node 62: 0.007834037009189733
Node 408: 0.007844556602368024
Node 424: 0.008719556651998127
